[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CLAIR-LAB-TECHNION/aidm/blob/main/notebooks/pddl.ipynb)

In [1]:
!pip install "aidm[pddl] @ git+https://github.com/CLAIR-LAB-TECHNION/aidm"

In [2]:
from aidm.environments.pddl.pddl_problem import PDDLProblem
from aidm.search import utils, defs
from aidm.search.best_first_search import best_first_search

In [3]:
DOMAIN = """
(define (domain n-table-blocks-world)
  (:requirements :strips :typing :equality)
  (:types block color table)
  (:predicates (on ?b1 ?b2 - block)
	           (ontable ?b - block ?t -table)
	           (clear ?b - block)
	           (handempty)
	           (holding ?b - block)
	           (bcolor ?b - block ?c -color)
 )

  (:action pick-up
	     :parameters (?b - block ?t - table)
	     :precondition (and (clear ?b)(ontable ?b ?t)(handempty))
	     :effect
	     (and (not (ontable ?b ?t))
		   (not (clear ?b))
		   (not (handempty))
		   (holding ?b)))

  (:action put-down
	     :parameters (?b - block ?t - table)
	     :precondition (holding ?b)
	     :effect
	     (and (not (holding ?b))
		   (clear ?b)
		   (handempty)
		   (ontable ?b ?t)))
		   
  (:action stack
	     :parameters (?b1 ?b2 - block)
	     :precondition (and (holding ?b1) (clear ?b2))
	     :effect
	     (and (not (holding ?b1))
		   (not (clear ?b2))
		   (clear ?b1)
		   (handempty)
		   (on ?b1 ?b2)))
		   
  (:action unstack
	     :parameters (?b1 ?b2 - block)
	     :precondition (and (on ?b1 ?b2) (clear ?b1)(handempty))
	     :effect
	     (and (holding ?b1)
		   (clear ?b2)
		   (not (clear ?b1))
		   (not (handempty))
		   (not (on ?b1 ?b2)))))
		   
"""

In [4]:
PROBLEM = """
(define (problem 3t5b)
(:domain n-table-blocks-world)
(:objects 
B1 B2 B3 B4 B5 - block
Y B G R P - color
T1 T2 T3  - table
)
(:init
(handempty)

(ontable B1 T1)
(clear B1)
(bcolor B1 Y)

(ontable B2 T1)
(clear B2)
(bcolor B2 B)

(ontable B3 T2)
(clear B3)
(bcolor B3 Y)

(ontable B4 T3)
(clear B4)
(bcolor B4 G)

(ontable B5 T3)
(clear B5)
(bcolor B5 R)

)
(:goal (and
(ontable B1 T3)
(ontable B2 T2)
))
)
"""

In [5]:
problem = PDDLProblem(domain_file=DOMAIN, problem_file=PROBLEM)

In [6]:
[
    best_value,
    best_node,
    best_plan,
    explored_count,
    ex_terminated
 ] = best_first_search(
    problem=problem,
    frontier=utils.FIFOQueue(),
    closed_list=utils.ClosedListOfKeys(),
    termination_criteria=utils.TerminationCriteriaGoalStateReached(),
    evaluation_criteria=utils.EvaluationCriteriaGoalCondition(),
    prune_func=None,
    constraints=None,
    log=True,
    log_file=None,
    iter_limit=defs.NA,
    time_limit=defs.NA
)

best_first_design(node) node number 1 cur_node:<Node State(literals=frozenset({clear(b3:block), bcolor(b3:block,y:color), ontable(b1:block,t1:table), clear(b5:block), bcolor(b4:block,g:color), clear(b1:block), ontable(b2:block,t1:table), ontable(b3:block,t2:table), clear(b4:block), clear(b2:block), bcolor(b1:block,y:color), ontable(b4:block,t3:table), bcolor(b2:block,b:color), ontable(b5:block,t3:table), handempty(), bcolor(b5:block,r:color)}), objects=frozenset({b1:block, b:color, t1:table, b4:block, p:color, r:color, t3:table, t2:table, y:color, b5:block, b3:block, b2:block, g:color}), goal=AND[ontable(b1:block,t3:table), ontable(b2:block,t2:table)])>
InMethod best_first_design(node): explored_count:1 cur_node:<Node State(literals=frozenset({clear(b3:block), bcolor(b3:block,y:color), ontable(b1:block,t1:table), clear(b5:block), bcolor(b4:block,g:color), clear(b1:block), ontable(b2:block,t1:table), ontable(b3:block,t2:table), clear(b4:block), clear(b2:block), bcolor(b1:block,y:color),

In [7]:
best_plan

['pick-up(b1:block,t1:table)',
 'put-down(b1:block,t3:table)',
 'pick-up(b2:block,t1:table)',
 'put-down(b2:block,t2:table)']